In [1]:
import re
from datetime import datetime, timedelta
from pathlib import Path
from typing import Optional

import pandas as pd
import numpy as np
import seaborn as sns
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle

plt.style.use('seaborn-whitegrid')

## Базовая реструктуризация данных

In [2]:
data_dir = Path('../data/2021_06_02/').resolve()
data_dir

PosixPath('/Users/yuralytkin/Development/work/itmo/rzd-fares-analysis/data/2021_06_02')

In [3]:
def load_train_data(
        train_num: str,
        train_class: str,
) -> Optional[pd.DataFrame]:
    places_fpath = data_dir / f'{train_num}-{train_class}-place.csv'
    prices_fpath = data_dir / f'{train_num}-{train_class}-price.csv'

    if not all(fpath.exists() for fpath in [places_fpath, prices_fpath]):
        return

    places = pd.read_csv(places_fpath, encoding='cp1251', sep=';', index_col=0).T
    prices = pd.read_csv(prices_fpath, encoding='cp1251', sep=';', index_col=0).T

    for df in [places, prices]:
        df.index = pd.to_datetime(df.index)
        df.index.name = 'date'
        df.columns = df.columns.map(int)
        df.columns.name = 'days'
        
    max_places = places.max().max()
    
    if np.isnan(max_places) or max_places == 0:
        return
    
    to_remove = places[places.max(axis=1) == 0].index
    
    for df in [places, prices]:
        df.drop(index=to_remove, inplace=True)
        
    places = places.reset_index().melt(id_vars='date', var_name='days', value_name='places')
    
    prices = prices.replace(0, np.nan)
    prices = prices.reset_index().melt(id_vars='date', var_name='days', value_name='price')

    train_data = places.merge(prices, how='outer', on=['date', 'days'])
    
    train_data = train_data.dropna(subset=['places', 'price'], how='all')
        
    train_data['num'] = train_num
    train_data['class'] = train_class
    train_data['train'] = f'{train_num}-{train_class}'

    return train_data

In [4]:
fname_re = re.compile('([^-]+)-([^-]+)-(price|place).csv')

trains = set()

for fpath in data_dir.iterdir():
    match = fname_re.fullmatch(fpath.name)
    
    if match is None:
        continue
        
    train_num, train_class, _ = match.groups()
    trains.add((train_num, train_class))
    
trains = sorted(trains)
    
data = pd.DataFrame()

for train_num, train_class in tqdm(trains):
    train_data = load_train_data(train_num, train_class)
    
    if train_data is not None:
        data = data.append(train_data, ignore_index=True)
        
data.head(2)

  0%|          | 0/376 [00:00<?, ?it/s]

,date,days,places,price,num,class,train
0,2021-04-03,1,62.0,3275.0,001А,Купе,001А-Купе
1,2021-04-04,1,33.0,3913.0,001А,Купе,001А-Купе


In [5]:
data['train'].nunique()

144

In [6]:
data.to_csv('../data/data_raw.csv', index=False, encoding='utf-8')

## Препроцессинг

In [7]:
data = pd.read_csv('../data/data_raw.csv', parse_dates=['date'], encoding='utf-8')

data.head(2)

,date,days,places,price,num,class,train
0,2021-04-03,1,62.0,3275.0,001А,Купе,001А-Купе
1,2021-04-04,1,33.0,3913.0,001А,Купе,001А-Купе


In [8]:
data['train'].nunique()

144

### Удаляем данные за праздники (29 апреля — 12 мая)

In [9]:
holidays = np.arange(
    datetime(year=2021, month=4, day=29),
    datetime(year=2021, month=5, day=13),
    timedelta(days=1),
)

data = data[~data['date'].isin(holidays)].copy()

data.head(2)

,date,days,places,price,num,class,train
0,2021-04-03,1,62.0,3275.0,001А,Купе,001А-Купе
1,2021-04-04,1,33.0,3913.0,001А,Купе,001А-Купе


### Заполненность строк для разных поездов

In [10]:
dates = data['date'].drop_duplicates().sort_values().values

split_id = np.where(dates > holidays[-1])[0][0]

In [11]:
def date_formatter(dt: datetime) -> str:
    return dt.strftime('%b %d (%a)')

In [12]:
filled_frac = pd.DataFrame({
    train: data[(data['train'] == train) & (data['days'] <= 30)]\
                    .pivot(index='date', columns='days', values='places')\
                    .notna().mean(axis=1)
    for train in sorted(data['train'].unique())
}).reindex(dates).fillna(0)
filled_frac.index = filled_frac.index.map(date_formatter)

filled_frac.head(2)

,001А-Купе,001А-СВ,002А-Купе,002А-СВ,003А-Купе,003А-СВ,004А-Купе,004А-СВ,005А-Купе,005А-СВ,...,778А-C1,778А-C2,779А-B1,779А-B2,779А-C1,779А-C2,780А-B1,780А-B2,780А-C1,780А-C2
date,,,,,,,,,,,,,,,,,,,,,
Apr 03 (Sat),0.333333,0.333333,0.333333,0.333333,0.000000,0.000000,0.000000,0.000000,0.333333,0.333333,...,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333
Apr 04 (Sun),0.366667,0.366667,0.366667,0.366667,0.366667,0.366667,0.366667,0.366667,0.366667,0.366667,...,0.366667,0.366667,0.366667,0.366667,0.366667,0.366667,0.366667,0.366667,0.366667,0.366667


In [13]:
ax = plt.figure(figsize=(20, 8)).gca()
sns.heatmap(filled_frac, vmin=0, vmax=1, ax=ax, xticklabels=True, yticklabels=True,
            cmap='coolwarm', cbar_kws={'aspect': 40})

ax.axhline(split_id, c='k', ls='dashed', lw=1)

for i in np.arange(0, filled_frac.shape[0], 7)[1:]:
    ax.axhline(i, c='k', ls='dotted', lw=0.8)

ax.set_xlabel('train')
ax.tick_params(axis='both', which='major', labelsize=6)

ax.set_title('Fraction of filled values for different trains\n'
             '(30-day lag)')

plt.savefig('../data/figures/filled_frac.pdf', transparent=True, bbox_inches='tight')
plt.close()

### Удаляем поезда, по которым слишком мало данных

__Важно!__ Надо не забыть, что мы не просто выкидываем данные, а планируем рассмотреть их отдельно, когда данных станет больше.

In [14]:
train_nums_to_remove = ['003А', '004А', '761А', '762А', '763А', '764А', '765А', '766А']

data = data[~data['num'].isin(train_nums_to_remove)].copy()

data['train'].nunique()

116

### Вместимость каждого поезда в зависимости от даты

In [15]:
capacity = data.groupby(['train', 'date'])['places'].max().reset_index()\
    .pivot(index='date', columns='train', values='places')\
    .reindex(dates).fillna(0)
capacity /= capacity.max()
capacity.index = capacity.index.map(date_formatter)

capacity.head(2)

train,001А-Купе,001А-СВ,002А-Купе,002А-СВ,005А-Купе,005А-СВ,006А-Купе,006А-СВ,701Н-СВ,701Н-Сидячий,...,778А-C1,778А-C2,779А-B1,779А-B2,779А-C1,779А-C2,780А-B1,780А-B2,780А-C1,780А-C2
date,,,,,,,,,,,,,,,,,,,,,
Apr 03 (Sat),0.750000,0.867188,0.652778,0.859375,0.577023,0.607143,0.347990,0.821429,0.952381,0.806268,...,0.931818,0.870270,1.000000,0.918367,1.0,0.809783,1.000000,0.877551,0.909091,0.829268
Apr 04 (Sun),0.564286,0.820312,0.777778,0.773438,0.469974,0.535714,0.488693,0.750000,0.857143,0.820513,...,0.954545,0.716216,0.823529,0.244898,0.5,0.171196,0.764706,0.448980,1.000000,0.471545


In [16]:
ax = plt.figure(figsize=(20, 8)).gca()
sns.heatmap(capacity, ax=ax, xticklabels=True, yticklabels=True,
            cmap='coolwarm', cbar_kws={'aspect': 40})

ax.axhline(split_id, c='k', ls='dashed', lw=1)

for i in np.arange(0, filled_frac.shape[0], 7)[1:]:
    ax.axhline(i, c='k', ls='dotted', lw=0.8)

ax.set_xlabel('train')
ax.tick_params(axis='both', which='major', labelsize=6)

ax.set_title('Capacity of different trains\n'
             '(relative to the maximal capacity over observed days)')

plt.savefig('../data/figures/capacity.pdf', transparent=True, bbox_inches='tight')
plt.close()

### Удаляем для некоторых поездов даты, в которые количество вагонов существенно отличается

In [17]:
mask = data['train'].isin(['005А-Купе', '006А-Купе'])\
    & (data['date'].dt.weekday == 4)

print(mask.sum())

data = data[~mask].copy()

463


In [18]:
mask = (data['train'] == '005А-Купе')\
    & (data['date'].dt.weekday == 6)

print(mask.sum())

data = data[~mask].copy()

250


In [19]:
mask = data['num'].isin(['705Н', '707Н'])\
    & data['class'].str.startswith('Сидячи')\
    & (data['date'].dt.weekday == 6)

print(mask.sum())

data = data[~mask].copy()

491


In [20]:
mask = (data['train'] == '773А-B2') & data['date'].dt.weekday.isin([5, 6])

print(mask.sum())

data = data[~mask].copy()

402


In [21]:
mask = (data['train'].isin(['752А-B2', '757А-B2', '760А-B2', '774А-B2']))\
    & data['date'].dt.weekday.isin([4, 5])

print(mask.sum())

data = data[~mask].copy()

1476


In [22]:
mask = (data['num'] == '757А') & data['date'].dt.weekday.isin([3, 4])

print(mask.sum())

data = data[~mask].copy()

1595


In [23]:
mask = (data['num'] == '774А') & (data['date'].dt.weekday == 6)

print(mask.sum())

data = data[~mask].copy()

919


In [24]:
data['train'].nunique()

116

### Данные по дням недели

In [25]:
data['weekday'] = data['date'].dt.weekday
data['day_name'] = data['date'].dt.day_name()

weekdays = data[['weekday', 'day_name']].drop_duplicates().sort_values(by='weekday')\
    .set_index('weekday')['day_name'].to_dict()

weekdays

{0: 'Monday',
 1: 'Tuesday',
 2: 'Wednesday',
 3: 'Thursday',
 4: 'Friday',
 5: 'Saturday',
 6: 'Sunday'}

In [26]:
trains = data['train'].drop_duplicates().sort_values().values
len(trains)

116

In [27]:
ncols = 6
nrows = len(trains) // ncols + (len(trains) % ncols > 0)

fig, axes = plt.subplots(ncols=ncols, nrows=nrows)
fig.set_size_inches(3 * ncols, 3 * nrows)
fig.subplots_adjust(wspace=0.4, hspace=0.5)
axes = axes.flatten()

for train, ax in zip(trains, axes):
    train_data = data[data['train'] == train]\
        .pivot(index='date', columns='days', values='places')
    
    for date, row in train_data.iterrows():
        weekday = date.weekday()
        
        _, labels = ax.get_legend_handles_labels()
        
        label = weekdays[weekday]
        
        if label in labels:
            label = None
        
        ax.plot(row, c=f'C{weekday}', lw=0.5, label=label)
    
    ax.set_title(train)
    ax.invert_xaxis()
    
for i in range(1, nrows, 3):
    axes[(i + 1) * ncols - 1].legend(loc='upper left', bbox_to_anchor=(1, 1))

for i in range(len(trains), len(axes)):
    axes[i].axis('off')
    
plt.savefig('../data/figures/places_vs_weekday.pdf', transparent=True, bbox_inches='tight')
plt.close()

No handles with labels found to put in legend.


### Точечные исправления данных, где это возможно

Несколько явно мусорных значений.

In [28]:
for train in ['757А-C1', '770А-C1']:
    for date, days in [
        (datetime(year=2021, month=5, day=16), 32),
        (datetime(year=2021, month=5, day=23), 39),
    ]:
        mask = (data['train'] == train)\
            & (data['date'] == date)\
            & (data['days'] == days)
        
        data.loc[mask, 'places'] = np.nan

In [29]:
for train in ['757А-C1', '757А-C1']:
    mask = (data['train'] == train)\
        & (data['date'] == datetime(year=2021, month=4, day=14))\
        & (data['days'] == 1)
    
    data.loc[mask, 'places'] = np.nan

In [30]:
data = data.dropna(subset=['places', 'price'], how='all')

Есть несколько поездов, в которых число вагонов явно увеличивалось извне (т.е. со стороны РЖД) в ходе приближения даты отправления. Такие поезда пока выкидываем, потому что это особенность, с которой пока не ясно, как работать.

In [31]:
trains_to_remove = ['001А-СВ', '002А-СВ', '005А-Купе', '006А-Купе']

data = data[~data['train'].isin(trains_to_remove)].copy()

In [32]:
mask = (data['train'].isin(['774А-B1', '774А-C1', '774А-C2']))\
    & data['date'].dt.weekday.isin([3, 4])

print(mask.sum())

data = data[~mask].copy()

1389


In [33]:
trains = data['train'].drop_duplicates().sort_values().values
len(trains)

112

In [34]:
ncols = 6
nrows = len(trains) // ncols + (len(trains) % ncols > 0)

fig, axes = plt.subplots(ncols=ncols, nrows=nrows)
fig.set_size_inches(3 * ncols, 3 * nrows)
fig.subplots_adjust(wspace=0.4, hspace=0.5)
axes = axes.flatten()

for train, ax in zip(trains, axes):
    train_data = data[data['train'] == train]\
        .pivot(index='date', columns='days', values='places')
    
    for date, row in train_data.iterrows():
        weekday = date.weekday()
        
        _, labels = ax.get_legend_handles_labels()
        
        label = weekdays[weekday]
        
        if label in labels:
            label = None
        
        ax.plot(row, c=f'C{weekday}', lw=0.5, label=label)
    
    ax.set_title(train)
    ax.invert_xaxis()
    
for i in range(1, nrows, 3):
    axes[(i + 1) * ncols - 1].legend(loc='upper left', bbox_to_anchor=(1, 1))

for i in range(len(trains), len(axes)):
    axes[i].axis('off')
    
plt.savefig('../data/figures/places_vs_weekday_clean.pdf', transparent=True, bbox_inches='tight')
plt.close()

In [36]:
for train in data['train'].unique():
    mask = data['train'] == train
    data.loc[mask, 'places_frac'] = data.loc[mask, 'places'] / data.loc[mask, 'places'].max()
    
data.head(2)

,date,days,places,price,num,class,train,weekday,day_name,places_frac
0,2021-04-03,1,62.0,3275.0,001А,Купе,001А-Купе,5,Saturday,0.442857
1,2021-04-04,1,33.0,3913.0,001А,Купе,001А-Купе,6,Sunday,0.235714


In [37]:
data.to_csv('../data/data.csv', index=False, encoding='utf-8')